In [5]:
# Импорт необходимх библиотек
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords


from string import punctuation
from collections import Counter

import pymorphy3

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier

In [6]:
# Загружаем данные
df_banks = pd.read_csv("C:\\Users\\Mr0Wo\\OneDrive\\Рабочий стол\\Git_repositories\\Data-science-cases\\ML-cases\\Text-classification\\Review-classification\\banks.csv", sep='\t', index_col='idx')

In [7]:
df_banks.head()

,Score,Text
idx,,
0,Positive,В Альфа-Банке работает замечательная девушка -...
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,Positive,Очень порадовала оперативность работы в банке....
3,Negative,Имела неосторожность оформить потреб. кредит в...
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...


In [8]:
# Проверяем баланс классов
df_banks['Score'].value_counts(normalize=True)

Positive    0.500036
Negative    0.499964
Name: Score, dtype: float64

In [9]:
# Делаем маппинг, меняя Positive на 1, а Negative на 0
df_banks['Score'] = df_banks['Score'].map({'Positive':1, 'Negative':0})

In [10]:
# Создаем токенайзен
morph = pymorphy3.MorphAnalyzer()
ru_stopwords = stopwords.words("russian")
def tokenizer(text):
    lemma_tokens = []  # Массив для обработанных токенов
    tokens = nltk.word_tokenize(text.lower())  # Текст превращаем в токены, заодно переводя в нижний регистра
    tokens = [token for token in tokens if token not in punctuation]  # Убираем все знаки пунктуации
    tokens = [token for token in tokens if token not in ru_stopwords]  # Убираем стоп-слова
    tokens = [token for token in tokens if token not in ['.', '..', '...', '....']]  # Убираем многоточия
    for token in tokens:  # Цикл, в котором мы проходимся по всем токенам в предложении
        token = morph.parse(token)[0].normal_form  # Лематизируем (приводим к начальной форме) токены
        lemma_tokens.append(token)  # Добавляем подготовленные токены в массив
    return lemma_tokens

In [11]:
# Проверка работы функции
tokenizer(df_banks.iloc[0]['Text'])

['альфа-банк',
 'работать',
 'замечательный',
 'девушка',
 'ильясова',
 'орный',
 'вежливый',
 'отзывчивый',
 'действительно',
 'участвовать',
 'запрос',
 'клиент',
 'приходить',
 'подряд',
 'ровно',
 'день',
 'каждый',
 'день',
 'помнить',
 'время',
 'мой',
 'приход',
 'помочь',
 'оформить',
 'кредит',
 'размер',
 '1млн',
 'рубль',
 'прийти',
 'партнёр',
 'передавать',
 'получить',
 'кредит',
 'покупать',
 'я',
 'авто',
 'специалист',
 'ильясова',
 'орный',
 'помочь',
 'мы',
 'вывести',
 'сумма',
 'день',
 'это',
 'непросто',
 'сделка',
 'состояться',
 'остаться',
 'довольный',
 'знакомый',
 'конец',
 'добавить',
 'понять',
 'почему',
 'прислать',
 'это',
 'отделение',
 'девушка',
 'большой',
 'такой',
 'замечательный',
 'специалист',
 'приобретать',
 'программа',
 'здоровье',
 'вообще',
 'связать',
 'альфа-банк',
 'быть']

In [12]:
# Применяем токенайзер к нашим данным, а результат сохраняем в отдельный столбец
df_banks['Tokens'] = df_banks['Text'].apply(lambda row: tokenizer(row))

In [13]:
df_banks.head(10)

,Score,Text,Tokens
idx,,,
0,1,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,..."
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,..."
2,1,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан..."
3,0,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре..."
4,0,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан..."
5,1,В конце февраля оформила кредитную карту банка...,"[конец, февраль, оформить, кредитный, карта, б..."
6,0,Месяц назад взял автокредит. До этого ходили п...,"[месяц, назад, взять, автокредит, ходить, разн..."
7,0,"Не говоря про махинации с бонусами, остановлюс...","[говорить, махинация, бонус, остановиться, про..."
8,0,"День добрый, моя мама С-на В.А.,1932г рождения...","[день, добрый, мама, с-на, в.а.,1932г, рождени..."


In [14]:
# Создаем словарь, в котором подсчитывем кол-во появлений каждого слова
words = Counter()
for txt in df_banks['Tokens']: 
    words.update(txt)
len(words)

90824

In [15]:
words

Counter({'альфа-банк': 4359,
         'работать': 5517,
         'замечательный': 418,
         'девушка': 3742,
         'ильясова': 2,
         'орный': 2,
         'вежливый': 1815,
         'отзывчивый': 160,
         'действительно': 1149,
         'участвовать': 86,
         'запрос': 983,
         'клиент': 12681,
         'приходить': 2359,
         'подряд': 107,
         'ровно': 288,
         'день': 15729,
         'каждый': 2420,
         'помнить': 718,
         'время': 8957,
         'мой': 15645,
         'приход': 149,
         'помочь': 2302,
         'оформить': 3888,
         'кредит': 14490,
         'размер': 1319,
         '1млн': 1,
         'рубль': 4661,
         'прийти': 5239,
         'партнёр': 305,
         'передавать': 163,
         'получить': 7620,
         'покупать': 400,
         'я': 3455,
         'авто': 233,
         'специалист': 3228,
         'мы': 2020,
         'вывести': 119,
         'сумма': 8931,
         'это': 18561,
         'непро

In [16]:
# Словарь, отображающий слова в коды
word_to_index = dict()
# Словарь, отображающий коды в слова
index_to_word = dict()
# В цикле проходимся по отсортированному словарю, индексируем слова и наоборот
for i, word in enumerate(words.most_common(10000 - 2)):
    word_to_index[word[0]] = i + 2
    index_to_word[i + 2] = word[0]

In [17]:
word_to_index

{'банк': 2,
 'карта': 3,
 'это': 4,
 'день': 5,
 'мой': 6,
 'сотрудник': 7,
 'который': 8,
 'кредит': 9,
 'деньга': 10,
 'счёт': 11,
 'отделение': 12,
 'клиент': 13,
 'год': 14,
 'сказать': 15,
 'вопрос': 16,
 'свой': 17,
 'очень': 18,
 'время': 19,
 'сумма': 20,
 'кредитный': 21,
 'мочь': 22,
 'получить': 23,
 'офис': 24,
 'такой': 25,
 'проблема': 26,
 'заявление': 27,
 'договор': 28,
 'работа': 29,
 'платёж': 30,
 'банкомат': 31,
 'телефон': 32,
 'позвонить': 33,
 'месяц': 34,
 'документ': 35,
 'дать': 36,
 'ответ': 37,
 'решить': 38,
 'хотеть': 39,
 'обслуживание': 40,
 'звонить': 41,
 'ваш': 42,
 'работать': 43,
 'услуга': 44,
 'претензия': 45,
 'прийти': 46,
 'вклад': 47,
 '«': 48,
 '»': 49,
 'другой': 50,
 'звонок': 51,
 'номер': 52,
 'написать': 53,
 'один': 54,
 'ситуация': 55,
 'рубль': 56,
 'человек': 57,
 'минута': 58,
 'сделать': 59,
 'просто': 60,
 'говорить': 61,
 'средство': 62,
 'быть': 63,
 'альфа-банк': 64,
 'заявка': 65,
 'весь': 66,
 'срок': 67,
 'очередь': 68,
 '2

In [18]:
index_to_word

{2: 'банк',
 3: 'карта',
 4: 'это',
 5: 'день',
 6: 'мой',
 7: 'сотрудник',
 8: 'который',
 9: 'кредит',
 10: 'деньга',
 11: 'счёт',
 12: 'отделение',
 13: 'клиент',
 14: 'год',
 15: 'сказать',
 16: 'вопрос',
 17: 'свой',
 18: 'очень',
 19: 'время',
 20: 'сумма',
 21: 'кредитный',
 22: 'мочь',
 23: 'получить',
 24: 'офис',
 25: 'такой',
 26: 'проблема',
 27: 'заявление',
 28: 'договор',
 29: 'работа',
 30: 'платёж',
 31: 'банкомат',
 32: 'телефон',
 33: 'позвонить',
 34: 'месяц',
 35: 'документ',
 36: 'дать',
 37: 'ответ',
 38: 'решить',
 39: 'хотеть',
 40: 'обслуживание',
 41: 'звонить',
 42: 'ваш',
 43: 'работать',
 44: 'услуга',
 45: 'претензия',
 46: 'прийти',
 47: 'вклад',
 48: '«',
 49: '»',
 50: 'другой',
 51: 'звонок',
 52: 'номер',
 53: 'написать',
 54: 'один',
 55: 'ситуация',
 56: 'рубль',
 57: 'человек',
 58: 'минута',
 59: 'сделать',
 60: 'просто',
 61: 'говорить',
 62: 'средство',
 63: 'быть',
 64: 'альфа-банк',
 65: 'заявка',
 66: 'весь',
 67: 'срок',
 68: 'очередь',
 69

In [19]:
# Функция, которая заменяет слово на соответсвующий ему индекс в word_to_index
def text_to_sequence(txt, word_to_index):
    seq = []
    for word in txt:
        index = word_to_index.get(word, 1) # 1 означает неизвестное слово
        # Неизвестные слова не добавляем в выходную последовательность
        if index != 1:
            seq.append(index)
    return seq

In [20]:
# Применяем функцию text_to_sequence к данным и сохраняем результаты в колонке Sequence
df_banks['Sequence'] = df_banks['Tokens'].apply(lambda row: text_to_sequence(row, word_to_index))

In [21]:
df_banks

,Score,Text,Tokens,Sequence
idx,,,,
0,1,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,...","[64, 43, 916, 80, 205, 1864, 358, 2704, 410, 1..."
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,...","[268, 855, 1177, 3206, 3880, 2974, 7332, 172, ..."
2,1,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан...","[18, 1055, 910, 29, 2, 476, 184, 3, 628, 1765,..."
3,0,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре...","[118, 4805, 74, 2363, 9, 64, 18, 2658, 1286, 3..."
4,0,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан...","[419, 3839, 276, 88, 2, 648, 3, 48, 4881, 123,..."
...,...,...,...,...
13994,1,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро...","[402, 2316, 809, 5015, 8144, 7, 3090, 127, 442..."
13995,1,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ...","[363, 24, 9481, 2147, 202, 14, 24, 96, 142, 68..."
13996,1,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация...","[517, 110, 18, 993, 55, 38, 225, 234, 3, 46, 3..."


In [22]:
# Разбиваем данные на тренировочную и тестовую подвыборки
X_train, X_test, y_train, y_test = train_test_split(df_banks['Sequence'], df_banks['Score'],
                                                   random_state=42,
                                                   test_size=.2)

In [23]:
# Функия для векторизации 
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))  # Создаем нулевую матрицу, где кол-во слов в словаре - стобцы, а длинна 
    # последовательности - строки
    for i, sequence in enumerate(sequences):
        for index in sequence:
            results[i, index] += 1.  # Если слово встреается в последовательности, 
            # то увеличиваем его векторное представление на 1
    return results

In [24]:
# Применяем вышеуказанную функцию к нашим фичам (признакам)
X_train = vectorize_sequences(X_train, 10000)
X_test = vectorize_sequences(X_test, 10000)

# Baseline

In [25]:
# Создаем 3 базовые модели
LogReg = LogisticRegression(max_iter=500)
rf = RandomForestClassifier()
cb = CatBoostClassifier(verbose=100, eval_metric='AUC')

In [26]:
# Обучаем модели
LogReg.fit(X_train, y_train)
rf.fit(X_train, y_train)
cb.fit(X_train, y_train)

Learning rate set to 0.028902
0:	total: 247ms	remaining: 4m 6s
100:	total: 6.71s	remaining: 59.7s
200:	total: 21s	remaining: 1m 23s
300:	total: 41.3s	remaining: 1m 35s
400:	total: 51.4s	remaining: 1m 16s
500:	total: 57.9s	remaining: 57.7s
600:	total: 1m 4s	remaining: 42.7s
700:	total: 1m 10s	remaining: 30.2s
800:	total: 1m 16s	remaining: 19.1s
900:	total: 1m 23s	remaining: 9.19s
999:	total: 1m 29s	remaining: 0us


In [27]:
# Функция, для сохранения метрик качества моделей в DataFrame
models = ['LogisticRegression', 'RandomForest', 'CatBoost']
y_pred = [LogReg.predict(X_test), rf.predict(X_test), cb.predict(X_test)]
df_metrics = pd.DataFrame()
for i in range(len(models)):
    s = pd.Series({'accuracy_score': accuracy_score(y_test, y_pred[i]),
                   'precision_score': precision_score(y_test, y_pred[i]), 
                   'recall_score': recall_score(y_test, y_pred[i]), 
                   'f1_score': f1_score(y_test, y_pred[i]),
                   'roc_auc_score': roc_auc_score(y_test, y_pred[i])},
                    name = models[i])
    df_metrics[models[i]] = s

In [28]:
df_metrics

,LogisticRegression,RandomForest,CatBoost
accuracy_score,0.940000,0.919643,0.927500
precision_score,0.943764,0.942018,0.948949
recall_score,0.935668,0.894210,0.903503
f1_score,0.939698,0.917492,0.925668
roc_auc_score,0.939997,0.919625,0.927483
